In [11]:
# setup
import pandas as pd
import numpy as np
import random
import statsmodels.api as sm
import sys
from ppi_py import ppi_ols_ci, classical_ols_ci, ppi_ols_pointestimate

df = pd.read_csv("../Data/5_SurveySampleLLM.csv.gz")

Covs = ['PedPed', 'Barrier', 'CrossingSignal', 'NumberOfCharacters',
        'DiffNumberOFCharacters', 'LeftHand', 'Man', 'Woman', 'Pregnant',
        'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless',
        'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive',
        'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor',
        'MaleDoctor', 'Dog', 'Cat', 
        'Intervention'
        ]

sys.version

'3.11.4 (v3.11.4:d2340ef257, Jun  6 2023, 19:15:51) [Clang 13.0.0 (clang-1300.0.29.30)]'

In [2]:
# very few missing predicted values for the dependent variable
print("Number of NAs Saved: ",df["Saved"].isna().sum())
print("Number of NAs gpt4turbo_wp_Saved: ",df["gpt4turbo_wp_Saved"].isna().sum())
print("Number of NAs gpt4o_wp_Saved: ",df["gpt4o_wp_Saved"].isna().sum())
print("Number of NAs gpt35turbo0125_wp_Saved: ",df["gpt35turbo0125_wp_Saved"].isna().sum())

Number of NAs Saved:  0
Number of NAs gpt4turbo_wp_Saved:  0
Number of NAs gpt4o_wp_Saved:  6
Number of NAs gpt35turbo0125_wp_Saved:  2


In [3]:
# functions to calculate weights for conjoint experiment
def CalcTheoreticalInt(r):
    # this function is applied to each row (r)
    if r["Intervention"]==0:
        if r["Barrier"]==0:
            if r["PedPed"]==1: p = 0.48
            else: p = 0.32
            
            if r["CrossingSignal"]==0:   p = p * 0.48
            elif r["CrossingSignal"]==1: p = p * 0.2
            else: p = p * 0.32
        else: p = 0.2

    else: 
        if r["Barrier"]==0:
            if r["PedPed"]==1: 
                p = 0.48
                if r["CrossingSignal"]==0: p = p * 0.48
                elif r["CrossingSignal"]==1: p = p * 0.32
                else: p = p * 0.2
            else: 
                p = 0.2
                if r["CrossingSignal"]==0: p = p * 0.48
                elif r["CrossingSignal"]==1: p = p * 0.2
                else: p = p * 0.32
        else: p = 0.32  
    
    return(p)  
        
def calcWeightsTheoretical(profiles):
    
    p = profiles.apply(CalcTheoreticalInt, axis=1)

    weight = 1/p 

    return(weight) 

            

In [4]:
# function from PPI to calculate stats
def _ols_get_stats(
    pointest,
    X,
    Y,
    Yhat,
    X_unlabeled,
    Yhat_unlabeled,
    w=None,
    w_unlabeled=None,
    use_unlabeled=True,
):
    """Computes the statistics needed for the OLS-based prediction-powered inference.

    Args:
        pointest (ndarray): A point estimate of the coefficients.
        X (ndarray): Covariates for the labeled data set.
        Y (ndarray): Labels for the labeled data set.
        Yhat (ndarray): Predictions for the labeled data set.
        X_unlabeled (ndarray): Covariates for the unlabeled data set.
        Yhat_unlabeled (ndarray): Predictions for the unlabeled data set.
        w (ndarray, optional): Sample weights for the labeled data set.
        w_unlabeled (ndarray, optional): Sample weights for the unlabeled data set.
        use_unlabeled (bool, optional): Whether to use the unlabeled data set.

    Returns:
        grads (ndarray): Gradient of the loss function with respect to the coefficients.
        grads_hat (ndarray): Gradient of the loss function with respect to the coefficients, evaluated using the labeled predictions.
        grads_hat_unlabeled (ndarray): Gradient of the loss function with respect to the coefficients, evaluated using the unlabeled predictions.
        inv_hessian (ndarray): Inverse Hessian of the loss function with respect to the coefficients.
    """
    n = Y.shape[0]
    N = Yhat_unlabeled.shape[0]
    d = X.shape[1]
    w = np.ones(n) if w is None else w / np.sum(w) * n
    w_unlabeled = (
        np.ones(N)
        if w_unlabeled is None
        else w_unlabeled / np.sum(w_unlabeled) * N
    )

    hessian = np.zeros((d, d))
    grads_hat_unlabeled = np.zeros(X_unlabeled.shape)
    if use_unlabeled:
        for i in range(N):
            hessian += (
                w_unlabeled[i]
                / (N + n)
                * np.outer(X_unlabeled[i], X_unlabeled[i])
            )
            grads_hat_unlabeled[i, :] = (
                w_unlabeled[i]
                * X_unlabeled[i, :]
                * (np.dot(X_unlabeled[i, :], pointest) - Yhat_unlabeled[i])
            )

    grads = np.zeros(X.shape)
    grads_hat = np.zeros(X.shape)
    for i in range(n):
        hessian += (
            w[i] / (N + n) * np.outer(X[i], X[i])
            if use_unlabeled
            else w[i] / n * np.outer(X[i], X[i])
        )
        grads[i, :] = w[i] * X[i, :] * (np.dot(X[i, :], pointest) - Y[i])
        grads_hat[i, :] = (
            w[i] * X[i, :] * (np.dot(X[i, :], pointest) - Yhat[i])
        )

    inv_hessian = np.linalg.inv(hessian).reshape(d, d)
    return grads, grads_hat, grads_hat_unlabeled, inv_hessian

def _power_analysis_stats(grads, grads_hat, inv_hessian):
    grads_ = grads - grads.mean(axis=0)
    grads_hat_ = grads_hat - grads_hat.mean(axis=0)
    cov = inv_hessian @ (grads_[:,None,:] * grads_hat_[:,:,None]).mean(axis=0) @ inv_hessian
    var = inv_hessian @ (grads_[:,None,:]*grads_[:,:,None]).mean(axis=0) @ inv_hessian
    var_hat = inv_hessian @ (grads_hat_[:,None,:]*grads_hat_[:,:,None]).mean(axis=0) @ inv_hessian
    rhos_sq = np.diag(cov)**2/(np.diag(var)*np.diag(var_hat))
    sigmas_sq = np.diag(var)
    return rhos_sq, sigmas_sq

def _estimate_ppi_SE(n, N, rho_sq, var_Y):
    if N == np.inf:
        return np.sqrt(var_Y*(1-rho_sq)/n)
    if N == 0:
        return np.sqrt(var_Y/n)
    var_ppi = var_Y*(1-rho_sq*N/(n+N))/n
    return np.sqrt(var_ppi)

def _estimate_classical_SE(n, var_Y):
    return np.sqrt(var_Y/n)

In [5]:
# calculate amce for intervention
def compute_amce(data, x, y, alpha=0.05):

    # specify regression for swerve or stay in lane
    if x=="Intervention":
        
        # calculate weights
        data.loc[:,"weights"] = calcWeightsTheoretical(data)
    
        # drop rows with missing values on dependent variable
        dd = data.dropna(subset=y)

        # if X=1 characters die if AV serves, if X=0 characters if AV stays
        X = dd["Intervention"]
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])
    

    # specify regression for relationship to vehicle
    if x=="Barrier":

        # consider only dilemmas without legality and only pedestrians vs passengers
        data_sub = data.loc[(data["CrossingSignal"]==0) & (data["PedPed"]==0), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        
        # if X=1 passengers die and if X=0 pedestrians die
        X = dd["Barrier"]

        # recode to estimate the preference for pedestrians over passengers 
        X = 1 - X
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])

    
    # specify regression for legality
    if x=="CrossingSignal": 
        
        # consider dilemmas with legality and only pedestrians vs pedestrians
        data_sub = data.loc[(data["CrossingSignal"]!=0) & (data["PedPed"]==1), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)

        # if X=1 pedestrians cross on a green light, if X=2 pedestrians cross on a red light 
        X = dd["CrossingSignal"]

        # create dummy variable to estimate preference for pedestrians that cross legally (1) vs legally (0)
        X = 2 - X 
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])

    

    # Specify regressions for the remaining six attributes
    if x=="Utilitarian":
        
        # consider dilemmas that compare 'More' versus 'Less' characters
        data_sub = data.loc[(data["ScenarioType"]=="Utilitarian") & (data["ScenarioTypeStrict"]=="Utilitarian"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Utilitarian'})

        # create dummy variable to estimate the preference for sparing more characters
        X = (dd.loc[:,"Utilitarian"]=="More").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])


    if x=="Species":
        
        # consider dilemmas that compare humans versus animals 
        data_sub = data.loc[(data["ScenarioType"]=="Species") & (data["ScenarioTypeStrict"]=="Species"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Species'})

        # create dummy variable to estimate the preference for sparing humans
        X = (dd.loc[:,"Species"]=="Hoomans").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])
    

    if x=="Gender":
        
        # consider dilemmas that compare women versus men
        data_sub = data.loc[(data["ScenarioType"]=="Gender") & (data["ScenarioTypeStrict"]=="Gender"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Gender'})

        # create dummy variable to estimate the preference for sparing women
        X = (dd.loc[:,"Gender"]=="Female").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])


    if x=="Fitness":
        
        # consider dilemmas that compare fit characters versus those that are not
        data_sub = data.loc[(data["ScenarioType"]=="Fitness") & (data["ScenarioTypeStrict"]=="Fitness"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Fitness'})

        # create dummy variable to estimate the preference for sparing fit characters
        X = (dd.loc[:,"Fitness"]=="Fit").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])


    if x=="Age":
        
        # consider dilemmas that compare younger versus older characters
        data_sub = data.loc[(data["ScenarioType"]=="Age") & (data["ScenarioTypeStrict"]=="Age"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Age'})

        # create dummy variable to estimate the preference for sparing younger characters
        X = (dd.loc[:,"Age"]=="Young").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])

    
    if x=="Social Status":
        
        # consider dilemmas that compare high status versus low status characters
        data_sub = data.loc[(data["ScenarioType"]=="Social Status") & (data["ScenarioTypeStrict"]=="Social Status"), :].copy()

        # calculate weights
        data_sub.loc[:,"weights"] = calcWeightsTheoretical(data_sub)

        # drop rows with missing values on dependent variable
        dd = data_sub.dropna(subset=y)
        dd = dd.rename(columns = {'AttributeLevel': 'Social Status'})

        # create dummy variable to estimate the preference for sparing high status characters
        X = (dd.loc[:,"Social Status"]=="High").astype(int)
        X = sm.add_constant(X)

        # define model with standard errors clustered on UserID
        model = sm.WLS(dd[y], X, weights=dd["weights"])



    # fit model and extract estimates
    fit = model.fit(cov_type = 'cluster', cov_kwds = {'groups': dd["UserID"]})
    coef = fit.params[x]
    ci = fit.conf_int(alpha=alpha).loc[x]

    # store results
    res = pd.DataFrame({
        'x': [x],
        'y': [y],
        'pointest_pooled': [coef],
        'conf_low_pooled': [ci[0]],
        'conf_high_pooled': [ci[1]]
    })

    return(res)

    


In [6]:
# function to calculate amce with ppi 
def compute_amce_ppi(n_data, N_data, x, y, alpha=0.05):

    # specify regression for swerve or stay in lane
    if x=="Intervention":
        
        # calculate weights
        n_data.loc[:,"weights"] = calcWeightsTheoretical(n_data)
        N_data.loc[:,"weights"] = calcWeightsTheoretical(N_data)
    
        # drop rows with missing values on dependent variable
        n_dd = n_data.dropna(subset=y)
        N_dd = N_data.dropna(subset=y)

        # if X=1 characters die if AV serves, if X=0 characters if AV stays
        n_X = n_dd["Intervention"]               
        N_X = N_dd["Intervention"]

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights



    # specify regression for relationship to vehicle
    if x=="Barrier":

        # consider only dilemmas without legality and only pedestrians vs passengers
        n_data_sub = n_data.loc[(n_data["CrossingSignal"]==0) & (n_data["PedPed"]==0), :].copy()
        N_data_sub = N_data.loc[(N_data["CrossingSignal"]==0) & (N_data["PedPed"]==0), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)
        
        # if X=1 passengers die and if X=0 pedestrians die
        n_X = n_dd["Barrier"]
        N_X = N_dd["Barrier"]

        # recode to estimate the preference for pedestrians over passengers 
        n_X = 1 - n_X
        N_X = 1 - N_X

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights

    

    # specify regression for legality
    if x=="CrossingSignal": 
        
        # consider dilemmas with legality and only pedestrians vs pedestrians
        n_data_sub = n_data.loc[(n_data["CrossingSignal"]!=0) & (n_data["PedPed"]==1), :].copy()
        N_data_sub = N_data.loc[(N_data["CrossingSignal"]!=0) & (N_data["PedPed"]==1), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # if X=1 pedestrians cross on a green light, if X=2 pedestrians cross on a red light 
        n_X = n_dd["CrossingSignal"]
        N_X = N_dd["CrossingSignal"]

        # create dummy variable to estimate preference for pedestrians that cross legally (1) vs legally (0)
        n_X = 2 - n_X 
        N_X = 2 - N_X 

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights
    


    # Specify regressions for the remaining six attributes
    if x=="Utilitarian":
        
        # consider dilemmas that compare 'More' versus 'Less' characters
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Utilitarian") & (n_data["ScenarioTypeStrict"]=="Utilitarian"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Utilitarian") & (N_data["ScenarioTypeStrict"]=="Utilitarian"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)
        
        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Utilitarian'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Utilitarian'})

        # create dummy variable to estimate the preference for sparing more characters
        n_X = (n_dd.loc[:,"Utilitarian"]=="More").astype(int)
        N_X = (N_dd.loc[:,"Utilitarian"]=="More").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights



    if x=="Species":
        
        # consider dilemmas that compare humans versus animals 
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Species") & (n_data["ScenarioTypeStrict"]=="Species"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Species") & (N_data["ScenarioTypeStrict"]=="Species"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Species'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Species'})

        # create dummy variable to estimate the preference for sparing humans
        n_X = (n_dd.loc[:,"Species"]=="Hoomans").astype(int)
        N_X = (N_dd.loc[:,"Species"]=="Hoomans").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights

    

    if x=="Gender":
        
        # consider dilemmas that compare women versus men
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Gender") & (n_data["ScenarioTypeStrict"]=="Gender"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Gender") & (N_data["ScenarioTypeStrict"]=="Gender"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Gender'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Gender'})

        # create dummy variable to estimate the preference for sparing women
        n_X = (n_dd.loc[:,"Gender"]=="Female").astype(int)
        N_X = (N_dd.loc[:,"Gender"]=="Female").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights



    if x=="Fitness":
        
        # consider dilemmas that compare fit characters versus those that are not
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Fitness") & (n_data["ScenarioTypeStrict"]=="Fitness"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Fitness") & (N_data["ScenarioTypeStrict"]=="Fitness"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Fitness'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Fitness'})

        # create dummy variable to estimate the preference for sparing fit characters
        n_X = (n_dd.loc[:,"Fitness"]=="Fit").astype(int)
        N_X = (N_dd.loc[:,"Fitness"]=="Fit").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights



    if x=="Age":
        
        # consider dilemmas that compare younger versus older characters
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Age") & (n_data["ScenarioTypeStrict"]=="Age"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Age") & (N_data["ScenarioTypeStrict"]=="Age"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Age'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Age'})

        # create dummy variable to estimate the preference for sparing younger characters
        n_X = (n_dd.loc[:,"Age"]=="Young").astype(int)
        N_X = (N_dd.loc[:,"Age"]=="Young").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights


    
    if x=="Social Status":
        
        # consider dilemmas that compare high status versus low status characters
        n_data_sub = n_data.loc[(n_data["ScenarioType"]=="Social Status") & (n_data["ScenarioTypeStrict"]=="Social Status"), :].copy()
        N_data_sub = N_data.loc[(N_data["ScenarioType"]=="Social Status") & (N_data["ScenarioTypeStrict"]=="Social Status"), :].copy()

        # calculate weights
        n_data_sub.loc[:,"weights"] = calcWeightsTheoretical(n_data_sub)
        N_data_sub.loc[:,"weights"] = calcWeightsTheoretical(N_data_sub)

        # drop rows with missing values on dependent variable
        n_dd = n_data_sub.dropna(subset=y)
        N_dd = N_data_sub.dropna(subset=y)

        # rename column to extract coefficient from result
        n_dd = n_dd.rename(columns = {'AttributeLevel': 'Social Status'})
        N_dd = N_dd.rename(columns = {'AttributeLevel': 'Social Status'})

        # create dummy variable to estimate the preference for sparing high status characters
        n_X = (n_dd.loc[:,"Social Status"]=="High").astype(int)
        N_X = (N_dd.loc[:,"Social Status"]=="High").astype(int)

        # add intercept
        n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
        N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

        # gold standard data
        n_Y_human   = n_dd["Saved"].to_numpy()    # observed outcomes
        n_Y_silicon = n_dd[y].to_numpy()          # predicted outcomes
        n_weights = n_dd["weights"].to_numpy()    # define weights

        # unlabeled data
        N_Y_silicon = N_dd[y].to_numpy()          # predicted outcomes
        N_weights = N_dd["weights"].to_numpy()    # define weights


    # calculate point estimate
    pointest_ppi = ppi_ols_pointestimate(X=n_X, Y=n_Y_human, Yhat=n_Y_silicon, 
                                         X_unlabeled=N_X, Yhat_unlabeled=N_Y_silicon, 
                                         w=n_weights, w_unlabeled=N_weights)

    # calculate PPI confidence intervals
    lower_CI_ppi, upper_CI_ppi = ppi_ols_ci(X=n_X, Y=n_Y_human, Yhat=n_Y_silicon, 
                                            X_unlabeled=N_X, Yhat_unlabeled=N_Y_silicon, 
                                            w=n_weights, w_unlabeled=N_weights, alpha=alpha)

    # calculate OLS confidence intervals
    lower_CI_ols, upper_CI_ols = classical_ols_ci(X=n_X, Y=n_Y_human, w=n_weights, alpha=alpha)


    # calculate rho
    beta = sm.WLS(n_Y_human, n_X, weights=n_weights).fit().params

    grads, grads_hat, grads_hat_unlabeled, inv_hessian = _ols_get_stats(
        pointest=beta, 
        X=n_X,
        Y=n_Y_human,
        Yhat= n_Y_silicon,
        X_unlabeled=N_X,
        Yhat_unlabeled=N_Y_silicon,
        w=n_weights,
        w_unlabeled=N_weights,
        use_unlabeled=False)
    
    rho_sq, var_y = _power_analysis_stats(grads, grads_hat, inv_hessian)

    # create and return the output DataFrame
    output_df = pd.DataFrame({
        "y": y,                              
        "x": x,                              # Predictor variable (scenario attribute)
        "pointest_ppi": pointest_ppi[1],     # PPI point estimate
        "conf_low_ppi": lower_CI_ppi[1],     # The lower bound of the PPI confidence interval
        "conf_high_ppi": upper_CI_ppi[1],    # The upper bound of the PPI confidence interval
        "conf_low_ols": lower_CI_ols[1],     # The lower bound of the OLS confidence interval
        "conf_high_ols": upper_CI_ols[1],    # The upper bound of the OLS confidence interval
        "rho": np.sqrt(rho_sq[1])},          # The association between predictions and outcomes
        index=[0])
    
    return output_df 

In [7]:
pd.concat([compute_amce(df, x="Intervention", y="Saved"), 
           compute_amce(df, x="Barrier", y="Saved"), 
           compute_amce(df, x="Gender", y="Saved"), 
           compute_amce(df, x="Fitness", y="Saved"), 
           compute_amce(df, x="Social Status", y="Saved"), 
           compute_amce(df, x="CrossingSignal",y="Saved"),
           compute_amce(df, x="Age", y="Saved"),
           compute_amce(df, x="Utilitarian", y="Saved"),
           compute_amce(df, x="Species", y="Saved")
           ])      

,x,y,pointest_pooled,conf_low_pooled,conf_high_pooled
0,Intervention,Saved,0.068216,0.052464,0.083969
0,Barrier,Saved,0.164845,0.136728,0.192962
0,Gender,Saved,0.159646,0.126552,0.192741
0,Fitness,Saved,0.120809,0.085368,0.156249
0,Social Status,Saved,0.170991,0.079034,0.262948
0,CrossingSignal,Saved,0.336349,0.306916,0.365781
0,Age,Saved,0.481846,0.450556,0.513135
0,Utilitarian,Saved,0.573304,0.545087,0.601520
0,Species,Saved,0.645940,0.617086,0.674794


The AMCE estimates computed above are the same as those calculated with the functions by Awad et al. (2018), see R script `7_CalculateAMCE.R`.


|           label            |    dv  |  amce |   se  | conf.low | conf.high |
|----------------------------|--------|-------|-------|----------|-----------|
|   Intervention             | Saved  | 0.068 | 0.008 |    0.052 |     0.084 |
|        Barrier             | Saved  | 0.165 | 0.014 |    0.137 |     0.193 |
|            Law             | Saved  | 0.336 | 0.015 |    0.307 |     0.366 |
|         Gender             | Saved  | 0.160 | 0.017 |    0.127 |     0.193 |
|        Fitness             | Saved  | 0.121 | 0.018 |    0.085 |     0.156 |
|  Social Status             | Saved  | 0.171 | 0.047 |    0.079 |     0.263 |
|            Age             | Saved  | 0.482 | 0.016 |    0.451 |     0.513 |
| No. Characters             | Saved  | 0.573 | 0.014 |    0.545 |     0.602 |
|        Species             | Saved  | 0.646 | 0.015 |    0.617 |     0.675 |


Below we show the PPI point estimates with a large sample size of human responses which we expect to be very close to the AMCE estimates obtained by applying classical inference.

In [8]:
ids = df["ResponseID"].unique()
N = 100
n = len(ids) - N
random.seed(2024)

n_ids = random.sample(ids.tolist(), k=n)
N_ids = random.sample(list(set(ids) - set(n_ids)), k=N)

df_human = df[ df["ResponseID"].isin(n_ids) ]
df_silicon = df [ df["ResponseID"].isin(N_ids)]

models = ["gpt4turbo_wp_Saved"]#,"gpt4o_wp_Saved","gpt35turbo0125_wp_Saved"]

results2 = pd.DataFrame()
for model in models: 
    
    print("Model: ", model)
    results1 = pd.concat([
        compute_amce_ppi(df_human, df_silicon, x="Intervention", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="Barrier", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="Gender", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="Fitness", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="Social Status", y=model), 
        compute_amce_ppi(df_human, df_silicon, x="CrossingSignal",y=model),
        compute_amce_ppi(df_human, df_silicon, x="Age", y=model),
        compute_amce_ppi(df_human, df_silicon, x="Utilitarian", y=model),
        compute_amce_ppi(df_human, df_silicon, x="Species", y=model)
        ],ignore_index=True)
    
    results2 = pd.concat([results2, results1],ignore_index=True)
    
results2.to_csv("../Data/7_rho.csv", index=False)
results2

Model:  gpt4turbo_wp_Saved


,y,x,pointest_ppi,conf_low_ppi,conf_high_ppi,conf_low_ols,conf_high_ols,rho
0,gpt4turbo_wp_Saved,Intervention,0.069353,0.058451,0.080260,0.058650,0.080464,0.346935
1,gpt4turbo_wp_Saved,Barrier,0.164356,0.148062,0.180672,0.148280,0.180897,0.312802
2,gpt4turbo_wp_Saved,Gender,0.159258,0.134282,0.184186,0.134110,0.184020,0.269240
3,gpt4turbo_wp_Saved,Fitness,0.121561,0.094977,0.148154,0.095043,0.148229,0.301971
4,gpt4turbo_wp_Saved,Social Status,0.175281,0.102719,0.247636,0.102464,0.247312,0.142767
5,gpt4turbo_wp_Saved,CrossingSignal,0.336182,0.317522,0.354839,0.317411,0.354733,0.274719
6,gpt4turbo_wp_Saved,Age,0.482231,0.459207,0.505145,0.459568,0.505508,0.208118
7,gpt4turbo_wp_Saved,Utilitarian,0.573309,0.552186,0.594152,0.551923,0.593888,0.196408
8,gpt4turbo_wp_Saved,Species,0.645272,0.625505,0.665038,0.625506,0.665036,0.050259


In [9]:
# sample size human subjects
ns = [100,500]

# sample size silicon subjects
Ns = [25,30,40,50,60,70,80,90,100,
      125,150,175,200,300,325,350,375,400,425,450,475,500,
      600,650,700,750,800,850,900,950,1000,
      1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,  
      2100,2200,2300,2400,2500]

# predictions
Ys = ["gpt4turbo_wp_Saved"]#,"gpt4o_wp_Saved","gpt35turbo0125_wp_Saved"]

# structural attributes of scenarios
Xs_scenarios  = ['Intervention', 'Barrier','CrossingSignal']

# attributes of characters
Xs_characters = ['Gender','Fitness','Social Status','Age','Utilitarian','Species']

# all attributes
Xs = Xs_scenarios + Xs_characters

# number of repetitions for combinations of n and N
reps = 500

result = pd.DataFrame()

for y in Ys:
  print(f"Iterating over dependent variable: {y}")
  
  for x in Xs:

    for n in ns:
      print(f"    Predictor: {x} with human sample size {n}")
      
      for N in Ns:

        for r in range(reps):

          # subset to dilemmas with variation on structural attribute
          if x in Xs_scenarios:

              cnt = df.groupby("ResponseID")[x].nunique()
              ids = cnt[ cnt > 1].index.tolist()

          # subset to dilemmas with relevant character attribute
          if x in Xs_characters:

              ids = df.loc[ (df["ScenarioType"]==x) & (df["ScenarioTypeStrict"]==x), "ResponseID"].tolist()
          
          # skip current iteration if target n is larger than population
          if (len(ids) < n):
             continue 

          # sample dilemmas for human subjects sample
          n_ids = random.sample(ids, k=n)
          
          # get remaining dilemma ids to sample from
          remaining_ids = list(set(ids) - set(n_ids))

          # skip current iteration if target N is larger than population
          if (len(remaining_ids) < N):
             continue 
          
          # sample dilemmas for silicon subjects sample
          N_ids = random.sample(remaining_ids, k=N)

          # subset data
          df_human = df[ df["ResponseID"].isin(n_ids) ]
          df_silicon = df [ df["ResponseID"].isin(N_ids)]

          # pool human and silicon subject decisions
          df_pooled = pd.concat([df_human,df_silicon], ignore_index=True)

          # compute ppi acme
          ppi = compute_amce_ppi(n_data=df_human, N_data=df_silicon, x=x, y=y)
          
          # compute acme on pooled data
          pooled = compute_amce(data=df_pooled,x=x,y=y)

          # store data
          to_append = pd.merge(ppi, pooled, on=['x','y'], how='outer')
          to_append["n"] = n
          to_append["N"] = N
          
          result = pd.concat([result, to_append], ignore_index=True)
          del ppi 
          del pooled 
          del to_append

          
          

          
          

Iterating over dependent variable: gpt4turbo_wp_Saved
    Predictor: Intervention with human sample size 100
    Predictor: Intervention with human sample size 500
    Predictor: Barrier with human sample size 100
    Predictor: Barrier with human sample size 500
    Predictor: CrossingSignal with human sample size 100
    Predictor: CrossingSignal with human sample size 500
    Predictor: Gender with human sample size 100
    Predictor: Gender with human sample size 500
    Predictor: Fitness with human sample size 100
    Predictor: Fitness with human sample size 500
    Predictor: Social Status with human sample size 100
    Predictor: Social Status with human sample size 500
    Predictor: Age with human sample size 100
    Predictor: Age with human sample size 500
    Predictor: Utilitarian with human sample size 100
    Predictor: Utilitarian with human sample size 500
    Predictor: Species with human sample size 100
    Predictor: Species with human sample size 500


In [10]:
result.to_csv("../Data/7_ResultsPPI.csv.gz", compression="gzip", index=False)
result


,y,x,pointest_ppi,conf_low_ppi,conf_high_ppi,conf_low_ols,conf_high_ols,rho,pointest_pooled,conf_low_pooled,conf_high_pooled,n,N
0,gpt4turbo_wp_Saved,Intervention,-0.000443,-0.156620,0.151869,-0.192798,0.126948,0.605763,0.030290,-0.154602,0.215181,100,25
1,gpt4turbo_wp_Saved,Intervention,0.162225,0.003918,0.320318,0.007065,0.326981,0.300923,0.246888,0.070402,0.423373,100,25
2,gpt4turbo_wp_Saved,Intervention,0.192664,0.036907,0.348872,0.028092,0.342801,0.337780,0.189865,0.015239,0.364491,100,25
3,gpt4turbo_wp_Saved,Intervention,-0.009242,-0.167409,0.148323,-0.153806,0.163140,0.238589,0.039567,-0.151713,0.230848,100,25
4,gpt4turbo_wp_Saved,Intervention,0.096439,-0.062807,0.263261,-0.049051,0.277744,0.127821,0.105112,-0.080590,0.290815,100,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381438,gpt4turbo_wp_Saved,Species,0.593002,0.533323,0.652681,0.533338,0.653167,0.097963,0.843520,0.822089,0.864951,500,2500
381439,gpt4turbo_wp_Saved,Species,0.642661,0.585482,0.698490,0.583677,0.696192,0.036514,0.849121,0.828247,0.869995,500,2500
381440,gpt4turbo_wp_Saved,Species,0.660238,0.606107,0.714714,0.611417,0.720703,0.132984,0.838966,0.817325,0.860607,500,2500
381441,gpt4turbo_wp_Saved,Species,0.654094,0.597391,0.710683,0.592138,0.706529,0.162665,0.843640,0.821995,0.865284,500,2500
